In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cobaya.model import get_model
import healpy as hp
from astropy.io import fits
from scipy.special import psi
from scipy.special import polygamma

from IPython.display import display, Math
import matplotlib.pyplot as plt
#from matplotlib_inline.backend_inline import set_matplotlib_formats
from astropy.visualization import quantity_support
from tqdm import tqdm

quantity_support()
#set_matplotlib_formats('svg')
from matplotlib import cm
from cycler import cycler
plt.rc('legend', frameon = False)
plt.rc('figure', figsize = (7, 7/1.25))
plt.rc('font', size = 14)

l_max = 1300
nside = 512
f_sky = 0.85

In [5]:
#cmb plus noise
Cls = pd.read_csv("../../PolSpice/PolSpice_v03-07-05/cl_hp_apodize.dat", skiprows = [0], delim_whitespace=True, 
                  names = ["TT", "EE", "BB", "TE", "TB", "EB", "ET", "BT", "BE"])
Cl_tt = np.array(Cls["TT"])
Cl_ee = np.array(Cls["EE"])
Cl_bb = np.array(Cls["BB"])
Cl_te = np.array(Cls["TE"])
Cl_eb = np.array(Cls["EB"])
Cl_tb = np.array(Cls["TB"])

#wait is Cl^TE not equal to Cl^ET when there's a mask?

In [6]:
#just noise
noise = fits.open('/mn/stornext/d16/cmbco/bp/johanres/commander_camb/camb_data/tod_070_rms_c0001_k000797_nside512.fits')
noise_data = noise[1].data
noise_temp = noise_data['TEMPERATURE'].flatten()
noise_q    = noise_data['Q_POLARISATION'].flatten()
noise_u    = noise_data['U_POLARISATION'].flatten()

sigma_T = np.sqrt(np.mean(noise_temp**2))
sigma_P = np.sqrt(np.mean(noise_q**2))

N_l_T = sigma_T**2 * 4.*np.pi/(12.*nside**2)
N_l_P = sigma_P**2 * 4.*np.pi/(12.*nside**2)

In [7]:
#beam data and pixel window
beam = fits.open('Bl_TEB_npipe6v19_70GHzx70GHz.fits')[1]
pixel_window_T, pixel_window_P = np.array(hp.sphtfunc.pixwin(nside = nside, pol = True, lmax = l_max))

func_tt = (beam.data['T'][:l_max+1] * pixel_window_T)**2
func_ee = (beam.data['E'][:l_max+1] * pixel_window_P)**2
func_bb = (beam.data['B'][:l_max+1] * pixel_window_P)**2
func_te = pixel_window_T * pixel_window_P * beam.data['T'][:l_max+1] * beam.data['E'][:l_max+1]
func_eb = pixel_window_P**2 * beam.data['E'][:l_max+1] * beam.data['B'][:l_max+1]
func_tb = pixel_window_T * pixel_window_P * beam.data['T'][:l_max+1] * beam.data['B'][:l_max+1]

In [ ]:
#set up cut sky likelihood function
def mask_like(_self=None):
    ells = np.arange(l_max+1)
    
    Cls_camb = _self.provider.get_Cl(ell_factor = False, units = "muK2")
    camb_tt = Cls_camb['tt'][:l_max+1] * func_tt 
    camb_ee = Cls_camb['ee'][:l_max+1] * func_ee 
    camb_bb = Cls_camb['bb'][:l_max+1] * func_bb 
    camb_te = Cls_camb['te'][:l_max+1] * func_te 
    
    Cls_map = np.zeros((l_max+1, 3, 3))
    Cls_map_theo = np.zeros((l_max+1, 3, 3))
    for ell in range(l_max+1):
        Cls_map[ell] = np.array([[Cl_tt[ell], Cl_te[ell], Cl_tb[ell]],
                                 [Cl_te[ell], Cl_ee[ell], Cl_eb[ell]],
                                 [Cl_tb[ell], Cl_eb[ell], Cl_bb[ell]]])
        Cls_map_theo[ell] = np.array([[camb_tt[ell] + N_l_T, camb_te[ell],         0.                  ],
                                      [camb_te[ell],         camb_ee[ell] + N_l_P, 0.                  ],
                                      [0.,                   0.,                   camb_bb[ell] + N_l_P]])
        #any weird correlations? why don't I need to multiply TE by noise?
        
    #Compute the log-likelihood following section III E of Hamimeche and Lewis (2008)
    logp = 0
    for ell0 in range(2, l_max+1):
        for ell1 in range(2, l_max+1):
            logp += X_g[ell0].T * np.inv(M)[ell0][ell1] * X_g[ell1] 
            #?? there is a lot of linear algebra in this paper which ngl I do not quite understand
            #obviously I still have to define X and M
            #but right now I'm still not totally sure how this works
